In [11]:
# Dependencies
# ! pip install googletrans
# ! pip install google_trans_new

import pandas as pd
import numpy as np
from google_trans_new import google_trans_new # google translate api (https://github.com/lushan88a/google_trans_new)

import os
os.chdir("/Users/Tosan.Johnson/Personal Projects/Kaggle Projects/Predict Future Sales") # changing working directory
from IPython.core.interactiveshell import InteractiveShell # allows multiple outputs per cell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# Read in data
df_icats = pd.read_csv('data/item_categories.csv')
df_items = pd.read_csv('data/items.csv')
df_sales = pd.read_csv('data/sales_train.csv')
df_shops = pd.read_csv('data/shops.csv')


## Initial Data Read

In [13]:
# Getting shapes for the tables
data_vars = [df_icats, df_items, df_sales, df_shops]
data_names = ["item categories", "items",'sales_train','shops']
data = zip(data_names, data_vars)
{name: df.shape for name,df in data}

{'item categories': (84, 2),
 'items': (22170, 3),
 'sales_train': (2935849, 6),
 'shops': (60, 2)}

There are a couple columns in different tables that have columns in russian. I'm using google translator to convert them to english. 

In [54]:
""" 
Purpose - Appends a column that translates a column in a table from some language to english
data_col_dict - dictionary{col_name:dataframe}
RETURNS - List of dataframes with english columns appended to dataframe(s) 
"""
def translate_cols(data_col_dict):
    translator = google_trans_new.google_translator()
    df_list = []
    for col, df in data_col_dict.items():
        unique_elements = df[col].unique()
        translated_elements = [translator.translate(text,lang_tgt='en') for text in unique_elements] # for some reason can't figure out how to split on backslash
        rosetta_stone = dict(zip(unique_elements, translated_elements))
        df[f"{col}_english"] = df[col].replace(rosetta_stone)
        df_list.append(df)
    return df_list   

' \nPurpose - Appends a column that translates a column in a table from some language to english\ndata_col_dict - dictionary{col_name:dataframe}\nRETURNS - List of dataframes with english columns appended to dataframe(s) \n'

This broke down due to too many API calls. I'll have to use multiprocessing and rotate proxies (it worked for the smaller tables). 

In [55]:
df_icats_copy = df_icats.copy()
df_items_copy = df_items.copy()
df_shops_copy = df_shops.copy()

russian_cols = {'item_category_name':df_icats_copy, 'item_name': df_items_copy, 'shop_name': df_shops_copy} # cols that are in russian (across moultiple tables)
# russian_cols = {'item_category_name':df_icats_copy} # cols that are in russian (across moultiple tables)

df_en_list = translate_cols(russian_cols)
# I'm lazy
df_en_list[0].to_csv('item_categories_en.csv')
df_en_list[1].to_csv('items_en.csv')
df_en_list[2].to_csv('shops_en.csv')

google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [89]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   date              object 
 1   date_block_num    int64  
 2   shop_id           int64  
 3   item_id           int64  
 4   item_price        float64
 5   item_cnt_day      float64
 6   item_category_id  int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 179.2+ MB


In [63]:
df_icats.head()
df_items.head()
df_sales.head()
df_shops.head()


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


## Explore the Data

### Questions I have for the data
1. How many items are are in each category per shop?
* A bit unfeasible to look at this. Every store holds at least 37 categories.
2. Do shops sell each item for the same price?
* No. Different stores can have different prices. Prices can even differ within a store (miniscule differences) 
3. Do shops specialize in certain items/groups?
* This will take manual inspection, but until I update the translate function I can't look into this.
* Could be a good idea to group shops by the item groups they sell. If they have a certain number of groups oevrlapping, they can be grouped. Create an indicator for the group.
4. Are there any sales/discounts that could explain any difference in price? (data doesn't specify, but good to consider for relative error)
* Looking at Question 2, there might be some discount at play. It's not obvious, but could be an explanation.
5. From the column name, item_cnt_day, I'm not exactly sure what this column represents. 
* I'm going to assume its inventory count for that day. Other than that, it'll take an algorithm to show any real value/pattern with this var
6. Average price for a shop?
* Looks like it might be a powerful variable
7. How does average price change over time for a shop?
* There's some variation (that's probably explained in the change in inventory), but when you look at a specific item for a store, very little (except for outliers)
8. Do shops sell different items over time?
* Yes
9. What is the date_block_num variable?
* It looks like this is a month variable (starting from Jan 2013)
* Different shops have different time periods available
10. How do average prices change across item groups? Expensive groups vs. Cheap Groups indicator? Variance in price in a group.
* Theres a lot to look at here so I'm just gonna throw this stuff in the model first and see what it does. After that I can dive deeper.

In [93]:
df_month0 = df_sales[df_sales['date_block_num'] == 0] # using month 1 as an estimate

In [84]:
# Prep
df_sales.isnull().sum() # No null values
df_sales = df_sales.merge(df_items[['item_id','item_category_id']], on='item_id', how='left') # merge the item category column over to the sales table
df_month0 = df_sales[df_sales['date_block_num'] == 0] # using month 1 as an estimate

In [80]:
# Question 9

# len(df_sales[(df_sales['shop_id'] == 57)]['date'].unique())
""" Results - Date block var is the forward month count for a shop. Different shops have different dates available. I can look further if I want """

972

In [87]:
# Question 1 - Month view

# a = df_sales[df_sales['date_block_num'] == 1].groupby('shop_id').agg({'item_id':'nunique', 'item_category_id':'nunique'})
# a.sort_values('item_id')
""" Results - Each store carries a ton of different item categories. I'm not sure if the categories are related or not (Russian). They also carry 300 items(min) up to 3500(max) per month """

In [97]:
# Question 2 - Month view

# a = df_month0[df_month0['item_id'] == 3432]
# a['shop_id'].nunique() # number of stores with item 3432
# a['item_price'].nunique() # number of different prices between all the different stores for item 3432
# a
# b = a.groupby('shop_id').agg({'item_price':'nunique'}).reset_index() # number of different prices for item 3432 at a single store
# b
# c = a[a['shop_id'] == 2] # further inspect the difference in price at a single store
# c
# d = a.drop_duplicates(['shop_id','item_price']) # see the difference in price across all stores
# d.describe()
""" Results - In month 0, there are 45 shops that hold item #3432 and there are 17 unique prices for it. Prices can range between stores. After some further inspection
              it looks like a store may have slightly different prices for a single item. For shop 3, it has 3 different prices for item 3432. The difference betweem the prices are pretty
              miniscule ($0.25 or $0.50), so this could be an error or something else at play.  """

45

17

In [ ]:
# Question 5 - Month view

# df_month0['item_cnt_day'].value_counts() # check the prevelance of values
# a = df_month0[(df_month0['shop_id'] == 2) & (df_month0['item_id'] == 3432)] # looking at a single item at a single store
# a.sort_values('date')
# b = df_month0[df_month0['item_cnt_day'] == 5] # trying to find a store/item with item_cnt_day of some high number
# b
# c = df_month0[(df_month0['shop_id'] == 25) & (df_month0['item_id'] == 3432)] # checking if theres some pattern related to sequential days
# c.sort_values('date')
""" Result - There are both (+/-) values. Most values are overwhelmingly 1, and from there they sequentially increase in prevelance. I can't find any patterns looking at the variable,
             which means it was inputted with inherent value. I had a hunch that it might be a cumulative count for the days the items been in the store, but I proved that wrong. My guess, 
             is, its showing the the inventory. The (-) values are still throwing me for a loop. """

In [ ]:
# Question 6 - Month view

# a = df_month0.groupby('shop_id').agg({'item_id':'count', 'item_price':'mean', 'item_category_id':'nunique'}).reset_index()
# a.sort_values('item_id')
b = df_month0.copy()
b['avg_price_per_shop'] = df_month0.groupby('shop_id')['item_price'].transform('mean')
b
""" Result -  Not really a question, but i created the variable. Seems to be a good variable to pass through the model"""

In [119]:
# Question 10 - Month View

a = df_month0.groupby('item_category_id').agg({'item_price':'mean'}).reset_index() # avg price across all stores
# a.sort_values('item_price')
# b = df_month0.groupby('shop_id').agg({'item_category_id':'nunique', 'item_id':'count'}).reset_index() # looking at what shops have what groups.
# b.sort_values('item_category_id')
c = df_month0[df_month0['shop_id'] == 25].groupby('item_category_id').agg({'item_price':'mean'}).reset_index() # avg price across all stores
# c.sort_values('item_price')
d2 = a.merge(c, on='item_category_id', how='left')
d2['price_diff'] = d2['item_price_x'] - d2['item_price_y'] # x is avg price across all stores, y is avg price at store 25
d2.sort_values('price_diff')

d = (df_month0.groupby('item_category_id')
              .agg(
                  avg_item_group_price=pd.NamedAgg(column="item_price", aggfunc="mean"),
                  std_item_group_price=pd.NamedAgg(column="item_price", aggfunc="std")
                  )
              .reset_index()
    ) # avg price and std for item groups across all stores
# d.sort_values('std_item_group_price')


f = (df_month0[df_month0['shop_id'] == 25].groupby('item_category_id')
                                          .agg(
                                                avg_item_group_price=pd.NamedAgg(column="item_price", aggfunc="mean"),
                                                std_item_group_price=pd.NamedAgg(column="item_price", aggfunc="std")
                                              )
                                          .reset_index()
    ) # avg price and std for item groups in store 25 
# f.sort_values('std_item_group_price')

g = d.merge(f, on='item_category_id', how='left')
# g.sort_values('std_item_group_price_x')
g['diff_avg_price'] = g['avg_item_group_price_x'] - g['avg_item_group_price_y'] # difference in avg price between all stores and store 25
g['diff_std_price'] = g['std_item_group_price_x'] - g['std_item_group_price_y'] # difference in std price between all stores and store 25
g[['item_category_id','diff_avg_price','diff_std_price']].sort_values('diff_std_price')

""" Results - I first looked at the avg price of a group across all stores. The numbers looked fine, but I wanted to double check and find the avg price across groups at a single
              store. So I took the store with the most groups (checked the total # of items as well) and compared the averages. Looking at the difference in prices between each
              group in one store (25) vs across all the stores, it could be a good variable that we create for every store. After looking into the variance of price within the
              item groups, they are a little bit too high for my comfort across all stores. """

,item_category_id,item_price
47,71,9.500000
54,82,31.175439
55,83,83.582225
42,66,98.030844
0,0,148.000000
1,1,148.000000
51,77,185.392291
53,81,188.507143
29,46,229.000000
25,40,244.485942


In [ ]:
# question 8 - single shop
a = (df_sales[df_sales['shop_id'] == 25].groupby('date_block_num')
                                        .agg(
                                             items_change=pd.NamedAgg(column='item_id', aggfunc='sum'), 
                                             item_groups_change=pd.NamedAgg(column='item_category_id', aggfunc='sum')
                                            )
                                        .reset_index()
    ) # groups the data by month and sums the id's of each item and group. If the sums are different across months, the items or groups in the stor changed. This is not foolproof
# a.sort_values('date_block_num')

""" Results - Unless I did something wrong, it looks like the items offered every month changes. Even the groups too. Theres a better way to check this, but I'm a little lazy, plus it
              it would take a ton of work i think.  """

In [ ]:
# Question 7 - single shop

a = (df_sales[df_sales['shop_id'] == 25].groupby('date_block_num')
                                        .agg(
                                             avg_price=pd.NamedAgg(column='item_price', aggfunc='mean'), 
                                            )
                                        .reset_index()
    ) # groups the data by month and sums the id's of each item and group. If the sums are different across months, the items or groups in the stor changed. This is not foolproof
a['chng_in_price_prev_month'] = a['avg_price'] - a['avg_price'].shift(1)
# a.sort_values('date_block_num')

b = (df_sales[(df_sales['shop_id'] == 25) & (df_sales['item_id'] == 5822)].groupby('date_block_num')
                                                                          .agg(
                                                                                avg_price=pd.NamedAgg(column='item_price', aggfunc='mean'), 
                                                                              )
                                                                          .reset_index()
    ) # groups the data by month and sums the id's of each item and group. If the sums are different across months, the items or groups in the stor changed. This is not foolproof
b['chng_in_price_prev_month'] = b['avg_price'] - b['avg_price'].shift(1)
# b.sort_values('date_block_num')

# df_sales[(df_sales['shop_id'] == 25)]['item_id'].value_counts()

""" Result - There is some variation in a stores avg price over time. This is consisten with finding out that stores offer different items over time. How to deal with this I'm
             not sure yet. Might be worth looking at the avg price of an item over time as well. So I checked, and the variation is less for a single item. It has jumps, but 
             they look like outliers (I looked at different items and I coulnd't find an obvious pattern. """

## The Lab


In [148]:
# The Lab - work stuff out here and then get rid of it




,date_block_num,avg_price,chng_in_price_prev_month
0,0,999.000000,NaN
1,1,1049.000000,50.000000
2,2,1049.000000,0.000000
3,3,1033.200000,-15.800000
4,4,1049.000000,15.800000
5,5,1049.000000,0.000000
6,6,1049.000000,0.000000
7,7,1049.000000,0.000000
8,8,1049.000000,0.000000
9,9,1049.000000,0.000000


20949    800
5822     371
7894     303
16169    290
13071    289
        ... 
9926       1
10019      1
9634       1
9637       1
19962      1
Name: item_id, Length: 15367, dtype: int64

## Insights from Exploration 
I wrote down everything I saw and explored above, but overall I don't think there was anything too shocking. I got a couple of good ideas for what variables I could include (mostly averages, maybe condensed down into degrees of magnitude, also some grouping for shops), but for now I'm going to move on to building out the modelling pipeline. I want to compare the lift I get in the model by adding new variables, so I'm going to hold off creating the variables explicitly for now. I don't think I actually need to do any language conversion, but it'll defintiely be very useful to have in the future especially with the multiprocessing and proxy randomizer included so it can be used for entire datasets.